In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fname = "8spin_1m1"
n = 7
# data_all = np.loadtxt(f"../output_small/{fname}/{fname}_sep-output.p",dtype=np.double)
data_all = np.loadtxt(f"../output_small/control_data/p7.p",dtype=np.double)

data_pi = data_all[:4]
data_pij = data_all[4:]

In [ ]:
model_all = np.loadtxt(f"../output_small/{fname}/{fname}_sep-output-out.j",dtype=np.double)
model_pi = model_all[:4]
model_pij = model_all[4:]

In [ ]:


fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# for ax in axs:
#     ax.set_aspect('equal', 'box')

ax = axs[0]
ax.scatter(data_pi, model_pi)
ax.set_xlabel('Data Pi')
ax.set_ylabel('Model Pi')

ax = axs[1]
ax.scatter(data_pij, model_pij)
ax.set_xlabel('Data Pij')
ax.set_ylabel('Model Pij')

# for ax in axs:
#     ax.set_xlim(0, 1)
#     ax.set_ylim(-1, 1)


plt.show()
